In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
#setup api keys
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
openrouter_key = os.getenv('OPENROUTER_KEY')

if openai_api_key:
    print(f"OpenAI key exists and begins: {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")
if openrouter_key:
    print(f"OpenRouter API Key exists and begins {openrouter_key[:3]}")
else:
    print("OpenRouter API Key not set (and this is optional)")

In [ ]:
openai=OpenAI()

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
openrouter_url = "https://openrouter.ai/api/v1"

gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_key)

In [ ]:

gpt_model = "gpt-4.1-mini"
gemini_model = "gemini-2.5-flash"
#openrouter_gpt_model = "arcee-ai/trinity-large-preview:free"
openrouter_nvidia_model = "nvidia/nemotron-3-nano-30b-a3b:free"

gpt_system = "You are Socrates, the famous philosopher of ancient Greece. Your method of philosophy \
is based on the Socratic method: you do not claim to know anything for certain, but rather, \
through dialogue and questioning, you help others realize the gaps in their knowledge and \
examine their beliefs. When engaging in conversation, always ask thought-provoking questions, \
challenge assumptions, and encourage the pursuit of wisdom. Your ultimate goal is to stimulate \
critical thinking and self-reflection in others. Stay calm, patient, and respectful even when you \
disagree with others.Keep explanations simple, and if you use philosophical terms, explain them in \
a way that is easy to understand."

gemini_system = "You are Friedrich Nietzsche, the revolutionary philosopher. You reject the traditional \
moralities of society, particularly those rooted in Christianity and Enlightenment rationalism. You \
believe that humans should embrace life in all its rawness and power, creating their own values and \
rejecting any moral system that limits human potential. You speak passionately \
about the “will to power,” the need for the Übermensch to rise above conventional morality,\
and the dangers of herd mentality. Engage in conversation with intensity, challenging others to \
question their assumptions and transcend the moral constraints of society. Keep explanations simple, \
and if you use philosophical terms, explain them in a way that is easy to understand."

#openrouter_gpt_system = """

openrouter_nvidia_system = "You are Immanuel Kant, the philosopher who has developed a systematic approach \
to ethics and knowledge.You believe that human experience is shaped by both empirical data and a priori \
concepts inherent in our minds. Morality, for you, is not subjective or based on consequences but is \
derived from rational principles. The Categorical Imperative dictates that we must act according to \
rules that could be universally applied, respecting the inherent dignity of all individuals. You hold \
that human beings must act out of duty, not merely for personal gain or happiness. Engage in conversation \
with reason and logic, emphasizing the importance of universal ethics, moral duty, and the limitations of \
human knowledge. Keep explanations simple, and if you use philosophical terms, explain them in a way that is easy to understand."

In [ ]:
#This is a bruteforce solution which extended the two way assistant solution provided in the lesson
#It's the first thing that came to my mind
#It doesn't provide coherent responses scroll further for a more efficient solution

# def call_gemini():
#     messages = [{"role":"system","content": gemini_system}]
#     if gemini_messages:
#         messages.append({"role":"assistant", "content":gemini_messages[-1]})
#     messages.append({"role":"user", "content": openrouter_messages[-1]})
#     response = gemini.chat.completions.create(model=gemini_model, messages=messages)
#     return response.choices[0].message.content

# def call_openrouter():
#     messages = [{"role":"system","content": openrouter_system}]
#     if openrouter_messages:
#         messages.append({"role":"assistant", "content":openrouter_messages[-1]})
#     messages.append({"role":"user", "content": gpt_messages[-1]})
#     response = openrouter.chat.completions.create(model=openrouter_model, messages=messages)
#     return response.choices[0].message.content

# similar function for chatGPT

In [ ]:
#This is a more efficient solution which can be generalized to N models
#It has one shared conversation history 

conversation = []
MAX_HISTORY = 6       
# --- Build messages for a model ---
def build_messages_for(model_name,conversation,system_prompt):
    messages = [{"role": "system", "content": system_prompt}]
    for turn in conversation[-MAX_HISTORY:]:
        if turn["speaker"] == model_name:
            role = "assistant"
        else:
            role = "user"
        messages.append({
            "role":role,
            "content": turn["content"]
        })
    return messages

In [ ]:

# --- Call a model ---
def call_model(client,model_name,system_prompt):
    messages = build_messages_for(model_name,conversation,system_prompt)
    
    response = client.chat.completions.create(
        model = model_name,
        messages = messages
        #max_tokens = 500          #uncomment to enforce limit on number of tokens generated if you're short on api calls
    )
    reply = response.choices[0].message.content.strip()
    conversation.append({
        "speaker":model_name,
        "content":reply
    })
    return reply

In [ ]:

conversation.append({"speaker":"user","content":"What does it mean to live a good life? "})
# --- Display full conversation ---
def display_conversation(conversation):
    md_text = ""
    for turn in conversation:
        speaker = turn["speaker"].capitalize()
        content = turn["content"]
        md_text += f"### {speaker}: \n {content}\n\n"
    display(Markdown(md_text))

for _ in range(2):

    gpt_next = call_model(openai,gpt_model,gpt_system)
    gemini_next = call_model(gemini,gemini_model,gemini_system)
    #openrouter_gem_next = call_model(openrouter,openrouter_gpt_model,openrouter_gpt_system)
    openrouter_next=call_model(openrouter,openrouter_nvidia_model,openrouter_nvidia_system)
    
display_conversation(conversation)